# Interactive visualization with `bokeh`

Let's use the [bokeh](https://bokeh.org/) visualization library to explore the spectra.

In [ ]:
import numpy as np
import pandas as pd
import glob
import bokeh

In [ ]:
bokeh.__version__

Read in the model spectra same as before:

In [ ]:
df_native = pd.read_csv('../models/spectra/sp_t1000g178nc_m0.0.gz', 
                        skiprows=[0, 1], 
                        delim_whitespace=True, 
                        compression='gzip',
                        names=['wavelength', 'flux']
                       ).sort_values('wavelength').reset_index(drop=True)

nir_mask = (df_native.wavelength > 1.2) & (df_native.wavelength < 1.35)

## decimate the data:
downsample = 5 
df_nir = df_native[nir_mask].rolling(2, win_type='gaussian').mean(std=3).iloc[::downsample, :]

In [ ]:
df_nir.shape

We will build heavily off of the [interact](https://github.com/lightkurve/lightkurve/blob/main/src/lightkurve/interact.py) method from the [lightkurve](https://docs.lightkurve.org/) framework.

In [ ]:
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import (
    LogColorMapper,
    Slider,
    RangeSlider,
    Span,
    ColorBar,
    LogTicker,
    Range1d,
    LinearColorMapper,
    BasicTicker,
)
from bokeh.layouts import layout, Spacer
from bokeh.models.tools import HoverTool
from bokeh.models.widgets import Button, Div
from bokeh.models.formatters import PrintfTickFormatter

In [ ]:
from scipy.ndimage import gaussian_filter1d

In [ ]:
def create_interact_ui(doc):
    
    # Make the spectrum source
    spec_source = ColumnDataSource(
        data=dict(
            wavelength=df_nir.wavelength.values,
            flux=gaussian_filter1d(df_nir.flux.values, 1.0),
        )
    )
    
    fig = figure(
        title="Sonora Bobcat in Bokeh",
        plot_height=340,
        plot_width=600,
        tools="pan,wheel_zoom,box_zoom,tap,reset",
        toolbar_location="below",
        border_fill_color="whitesmoke",
    )
    fig.title.offset = -10
    fig.yaxis.axis_label = "Flux "
    fig.xaxis.axis_label = "Wavelength (micron)"

    fig.step(
            "wavelength",
            "flux",
            line_width=1,
            color="gray",
            source=spec_source,
            nonselection_line_color="gray",
            nonselection_line_alpha=1.0,
        )
    
    # Slider to decimate the data
    cadence_slider = Slider(
            start=1,
            end=200,
            value=1,
            step=1,
            title="Spectral resolution kernel",
            width=490,
        )
    
    def update_upon_decimation(attr, old, new):
        """Callback to take action when cadence slider changes"""
        #spec_source.data["wavelength"] = df_nir.wavelength.values[::new]
        spec_source.data["flux"] = gaussian_filter1d(df_nir.flux.values, new)
            
        
    cadence_slider.on_change("value", update_upon_decimation)
    
    widgets_and_figures = layout(
            [fig],
            [cadence_slider]
        )
    doc.add_root(widgets_and_figures)

In [ ]:
output_notebook(verbose=False, hide_banner=True)
show(create_interact_ui)

Woohoo, it works!